In [1]:

import matplotlib.pyplot as plt
import numpy as np
import sys

In [2]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation, Dense
from keras.layers import Flatten, Reshape
from keras.layers import Conv1D
from keras.layers import concatenate
from keras.optimizers import Adam, RMSprop
from keras.models import load_model


def random_batch(X_train, y_train, batch_size):
    index_set = np.random.randint(0, X_train.shape[0], batch_size)
    X_batch = X_train[index_set]
    y_batch = y_train[index_set]
    return X_batch, y_batch

In [3]:
#Model Structure
model_name = 'ANC_16'

#the standard parameters: message, key, and ciphertext bit lengths
m_bits = 16
k_bits = 16
c_bits = 16
pad = 'same'

# Compute the size of the message space
m_train = 2 ** (m_bits + k_bits)

alice_file = 'models/crypto/' + model_name + '-alice_16'
bob_file = 'models/crypto/' + model_name + '-bob_16'
eve_file = 'models/crypto/' + model_name + '-eve_16'
### Network arch
K.clear_session()
kersize = 4

##### Alice network #####
#
alice_input0 = Input(shape=(m_bits,))  #message
alice_input1 = Input(shape=(k_bits,))  #key
alice_input = concatenate([alice_input0, alice_input1], axis=1)

alice_dense1 = Dense(units=(m_bits + k_bits))(alice_input)
alice_dense1a = Activation('tanh')(alice_dense1)

alice_reshape = Reshape((m_bits + k_bits, 1,))(alice_dense1a)

alice_conv1 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(alice_reshape)
alice_conv1a = Activation('tanh')(alice_conv1)
alice_conv2 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(alice_conv1a)
alice_conv2a = Activation('tanh')(alice_conv2)
alice_conv3 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(alice_conv2a)
alice_conv3a = Activation('tanh')(alice_conv3)
alice_conv4 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(alice_conv3a)
alice_conv4a = Activation('tanh')(alice_conv4)

alice_flat = Flatten()(alice_conv4a)
alice_output = Dense(units=c_bits, activation='tanh')(alice_flat)  #ciphertext

alice = Model([alice_input0, alice_input1], alice_output, name='alice')
alice.summary()

##### Bob network #####
#
bob_input0 = Input(shape=(c_bits,))  #ciphertext
bob_input1 = Input(shape=(k_bits,))  #key
bob_input = concatenate([bob_input0, bob_input1], axis=1)

bob_dense1 = Dense(units=(c_bits + k_bits))(bob_input)
bob_dense1a = Activation('tanh')(bob_dense1)

bob_reshape = Reshape((c_bits + k_bits, 1,))(bob_dense1a)

bob_conv1 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(bob_reshape)
bob_conv1a = Activation('tanh')(bob_conv1)
bob_conv2 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(bob_conv1a)
bob_conv2a = Activation('tanh')(bob_conv2)
bob_conv3 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(bob_conv2a)
bob_conv3a = Activation('tanh')(bob_conv3)
bob_conv4 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(bob_conv3a)
bob_conv4a = Activation('tanh')(bob_conv4)

bob_flat = Flatten()(bob_conv4a)
bob_output = Dense(units=m_bits, activation='sigmoid')(bob_flat)  #decrypted message

bob = Model([bob_input0, bob_input1], bob_output, name='bob')
bob.summary()

# Eve network
eve_input = Input(shape=(c_bits,))  #ciphertext only

eve_dense1 = Dense(units=(c_bits + k_bits))(eve_input)
eve_dense1a = Activation('tanh')(eve_dense1)
eve_dense2 = Dense(units=(m_bits + k_bits))(eve_dense1a)
eve_dense2a = Activation('tanh')(eve_dense2)

eve_reshape = Reshape((m_bits + k_bits, 1,))(eve_dense2a)

eve_conv1 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(eve_reshape)
eve_conv1a = Activation('tanh')(eve_conv1)
eve_conv2 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(eve_conv1a)
eve_conv2a = Activation('tanh')(eve_conv2)
eve_conv3 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(eve_conv2a)
eve_conv3a = Activation('tanh')(eve_conv3)
eve_conv4 = Conv1D(filters=4, kernel_size=kersize, strides=1, padding=pad)(eve_conv3a)
eve_conv4a = Activation('tanh')(eve_conv4)

eve_flat = Flatten()(eve_conv4a)
eve_output = Dense(units=m_bits, activation='sigmoid')(eve_flat)  #code break attempt

eve = Model(eve_input, eve_output, name='eve')
eve.summary()
alice.compile(loss='mse', optimizer='sgd')
bob.compile(loss='mse', optimizer='sgd')
eve.compile(loss='mse', optimizer='sgd')
# if False:
#     alice.summary()
#     bob.summary()
#     eve.summary()
### Loss + Optimizer
# Establish the communication channels by linking inputs to outputs
#
aliceout = alice([alice_input0, alice_input1])
bobout = bob([aliceout, bob_input1])  # bob sees ciphertext AND key
eveout = eve(aliceout)  # eve doesn't see the key, only the cipher

eveloss = K.mean(K.sum(K.abs(alice_input0 - eveout), axis=-1))

#
bobloss = K.mean(K.sum(K.abs(alice_input0 - bobout), axis=-1))
abeloss = bobloss + K.square(m_bits / 2 - eveloss) / ((m_bits // 2) ** 2)

# Optimizer and compilation

abeoptim = Adam()
eveoptim = Adam()

# Build and compile the model, used for training Alice-Bob networks
#
abemodel = Model([alice_input0, alice_input1, bob_input1], bobout, name='abemodel')
abemodel.add_loss(abeloss)
abemodel.compile(optimizer=abeoptim)

# Build and compile the EVE model, used for training Eve net (with Alice frozen)
#
alice.trainable = False
evemodel = Model([alice_input0, alice_input1], eveout, name='evemodel')
evemodel.add_loss(eveloss)
evemodel.compile(optimizer=eveoptim)


Model: "alice"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 32)                   0         ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 dense (Dense)               (None, 32)                   1056      ['concatenate[0][0]']     

In [5]:
### Train / save / restore
# Keep track of loss at every iteration for the final graph
import numpy as np
import csv
import sys
abelosses = []
boblosses = []
evelosses = []
# Assuming you have binary_fields_1M.csv generated from the previous step
binary_file = 'binary_fields_1M.csv'

# Load binary data from the file
def load_binary_data(filename):
    data = []
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            field1, field2 = row
            data.append((field1, field2))
    return data

data = load_binary_data(binary_file)

n_epochs = 30
batch_size = 256
m_train = len(data)  # Assuming the length of data is m_train
n_batches = m_train // batch_size

abecycles = 1
evecycles = 2

epoch = 0
print("Training for", n_epochs, "epochs with", n_batches, "batches of size", batch_size)

while epoch < n_epochs:
    abelosses0 = []  # epoch-bound losses for text display during training
    boblosses0 = []
    evelosses0 = []
    for iteration in range(n_batches):
        
        # Train the A-B+E network
        for cycle in range(abecycles):
            # Select a random batch of messages, and a random batch of keys
            indices = np.random.randint(0, len(data), batch_size)
            m_batch = [data[i][0] for i in indices]
            k_batch = [data[i][1] for i in indices]
            # Convert binary strings to numpy arrays
            m_batch = np.array([[int(bit) for bit in msg] for msg in m_batch])
            k_batch = np.array([[int(bit) for bit in key] for key in k_batch])
            loss = abemodel.train_on_batch([m_batch, k_batch, k_batch], None)
        
        abelosses0.append(loss)
        abelosses.append(loss)
        abeavg = np.mean(abelosses0)
            
        # Evaluate Bob's ability to decrypt a message
        m_enc = alice.predict([m_batch, k_batch])
        m_dec = bob.predict([m_enc, k_batch])
        loss = np.mean(np.sum(np.abs(m_batch - m_dec), axis=-1))
        boblosses0.append(loss)
        boblosses.append(loss)
        bobavg = np.mean(boblosses0)
        
        # Train the EVE network
        for cycle in range(evecycles):
            indices = np.random.randint(0, len(data), batch_size)
            m_batch = [data[i][0] for i in indices]
            k_batch = [data[i][1] for i in indices]
            # Convert binary strings to numpy arrays
            m_batch = np.array([[int(bit) for bit in msg] for msg in m_batch])
            k_batch = np.array([[int(bit) for bit in key] for key in k_batch])
            loss = evemodel.train_on_batch([m_batch, k_batch], None)
        
        evelosses0.append(loss)
        evelosses.append(loss)
        eveavg = np.mean(evelosses0)
        
        if iteration % max(1, (n_batches // 100)) == 0:
            print("\rEpoch {:3}: {:3}% | abe: {:2.3f} | eve: {:2.3f} | bob: {:2.3f}".format(
                epoch, 100 * iteration // n_batches, abeavg, eveavg, bobavg), end="")
            sys.stdout.flush()
    
    print()
    epoch += 1
    
print('Training finished.')


Training for 30 epochs with 195 batches of size 256
8/8 [==============================] - 0s 4ms/step
Epoch   0:  99% | abe: 5.304 | eve: 5.296 | bob: 5.135
8/8 [==============================] - 0s 3ms/step
Epoch   1:  99% | abe: 2.361 | eve: 3.326 | bob: 2.001
8/8 [==============================] - 0s 3ms/step
Epoch   2:  99% | abe: 1.098 | eve: 3.538 | bob: 0.773
8/8 [==============================] - 0s 3ms/step
Epoch   3:  99% | abe: 0.769 | eve: 4.199 | bob: 0.528
8/8 [==============================] - 0s 3ms/step
Epoch   4:  99% | abe: 0.559 | eve: 4.699 | bob: 0.373
8/8 [==============================] - 0s 3ms/step
Epoch   5:  99% | abe: 0.340 | eve: 4.914 | bob: 0.184
8/8 [==============================] - 0s 3ms/step
Epoch   6:  99% | abe: 0.275 | eve: 5.194 | bob: 0.143
8/8 [==============================] - 0s 3ms/step
Epoch   7:  99% | abe: 0.230 | eve: 5.456 | bob: 0.120
8/8 [==============================] - 0s 4ms/step
Epoch   8:  99% | abe: 0.201 | eve: 5.633 | bob: 

In [ ]:
steps = -1
plt.figure(figsize=(7, 4))
plt.plot(abelosses[:steps], label='A-B', alpha=0.99)
plt.plot(evelosses[:steps], label='Eve', alpha=0.99)
plt.plot(boblosses[:steps], label='Bob', alpha=0.99)
plt.xlabel("Iterations", fontsize=13)
plt.ylabel("Loss", fontsize=13)
plt.legend(fontsize=13, loc='upper right')

plt.savefig("images/" + model_name + "-all.png", transparent=True)  #dpi=100
plt.show()
alice.save(alice_file + '.h5', overwrite=True)
bob.save(bob_file + '.h5', overwrite=True)
eve.save(eve_file + '.h5', overwrite=True)
alice = load_model(alice_file + '.h5')
bob = load_model(bob_file + '.h5')
eve = load_model(eve_file + '.h5')
### Evaluate
n_examples = 10000

m_batch = np.random.randint(0, 2, m_bits * n_examples).reshape(n_examples, m_bits)
k_batch = np.random.randint(0, 2, m_bits * n_examples).reshape(n_examples, m_bits)

m_enc = alice.predict([m_batch, k_batch])

m_dec = (bob.predict([m_enc, k_batch]) > 0.5).astype(int)
m_att = (eve.predict(m_enc) > 0.5).astype(int)

bdiff = np.abs(m_batch - m_dec)
bsum = np.sum(bdiff, axis=-1)
ediff = np.abs(m_batch - m_att)
esum = np.sum(ediff, axis=-1)

print("Bob % correct: ", 100.0 * np.sum(bsum == 0) / n_examples, '%')
print("Eve % correct: ", 100.0 * np.sum(esum == 0) / n_examples, '%')
alice = load_model('models/crypto/ANC-alice_16.h5')
bob = load_model('models/crypto/ANC-bob_16.h5')
eve = load_model('models/crypto/ANC-eve_16.h5')

